In [1]:
import PIL
import torch
import glob as gb
import numpy as np
from PIL import Image
import os
import random
import torch.nn as nn

model_path = '/home/angelo/Desktop/Github/Feature-Extractors/'
os.sys.path.append(model_path)

from numpy import dot
from numpy.linalg import norm
import matplotlib.pyplot as plt
import random

random.seed(1)
np.random.seed(1)

In [55]:
# Initialization of models

def initialize_senet50_2048():
    import senet50_ft_pytorch.senet50_ft_dims_2048 as model
    network = model.senet50_ft(weights_path=model_path + 'senet50_ft_pytorch/senet50_ft_dims_2048.pth')
    network.eval()
    return network

def initialize_senet50_256():
    import senet50_256_pytorch.senet50_256 as model
    network = model.senet50_256(weights_path=model_path + 'senet50_256_pytorch/senet50_256.pth')
    network.eval()
    return network

def initialize_senet50_128():
    import senet50_128_pytorch.senet50_128 as model
    network = model.senet50_128(weights_path=model_path + 'senet50_128_pytorch/senet50_128.pth')
    network.eval()
    return network

def initialize_resnet50_128():
    import resnet50_128_pytorch.resnet50_128 as model
    network = model.resnet50_128(weights_path=model_path + 'resnet50_128_pytorch/resnet50_128.pth')
    network.eval()
    return network

In [77]:
# hyper parameters

def cos_sim(a,b):
    return 1 - dot(a, b)/(norm(a)*norm(b))

def euc_dis(a,b):
    return norm(a-b)

def load_image(path='', resize=False):
    '''
    VGGFace2 Data has mean = (131.0912, 103.8827, 91.4953)
    and std = (1,1,1) with no scaling
    '''
    mean = (131.0912, 103.8827, 91.4953)
    img = Image.open(path)
    if resize:
        img = img.resize((224,224), resample=PIL.Image.BICUBIC)
    
    img = np.array(img) - mean # Normalizing data before inference
    return img
    
def load_data(path=''):
    '''
    VGGFace2 Data has mean = (131.0912, 103.8827, 91.4953)
    and std = (1,1,1)
    '''
    mean = (131.0912, 103.8827, 91.4953)
    shape = (224,224,3)
    
    short_size = 224.0
    crop_size = shape
    img = Image.open(path)
    
    im_shape = np.array(img.size)    # in the format of (width, height, *)
    img = img.convert('RGB')

    ratio = float(short_size) / np.min(im_shape)
    img = img.resize(size=(int(np.ceil(im_shape[0] * ratio)),   # width
                           int(np.ceil(im_shape[1] * ratio))),  # height
                           resample=PIL.Image.BICUBIC)

    x = np.array(img)  # image has been transposed into (height, width)
    newshape = x.shape[:2]
    h_start = (newshape[0] - crop_size[0])//2
    w_start = (newshape[1] - crop_size[1])//2
    x = x[h_start:h_start+crop_size[0], w_start:w_start+crop_size[1]]
    x = x - mean
    return x

def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

def image_encoding(model, facepaths, batch_size):
    print('==> compute image-level feature encoding.')
    num_faces = len(facepaths)
    face_feats = np.empty((num_faces, 128))
    imgpaths = facepaths
    imgchunks = list(chunks(imgpaths, batch_size))

    for c, imgs in enumerate(imgchunks):
        im_array = np.array([load_data(path=i, shape=(224, 224, 3)) for i in imgs])
        f = model(torch.Tensor(im_array.transpose(0, 3, 1, 2)))[1].detach().cpu().numpy()[:, :, 0, 0]
        start = c * batch_size
        end = min((c + 1) * batch_size, num_faces)
        # This is different from the Keras model where the normalization has been done inside the model.
        face_feats[start:end] = f / np.sqrt(np.sum(f ** 2, -1, keepdims=True))
    
    return face_feats

def return_embedding(model, image, embedding_size=128):
    face_feats = np.empty((1,embedding_size)) # Embedding size
    im_array = np.array(image).reshape((1,3,224,224))
    f = model(torch.Tensor(im_array))[1].detach().cpu().numpy()[:, :, 0, 0]
    face_feats = f / np.sqrt(np.sum(f ** 2, -1, keepdims=True)) # Normalizing embedding
    return face_feats.reshape(-1)

def return_activations(model, image):
    '''
    Currently it only accepts SeNet50 activations.
    '''
    activations = list()
    normalized_values = 0
    im_array = np.array(image).reshape((1,3,224,224))
    for i in range(2,7):
        f = model(torch.Tensor(im_array))[i].detach().cpu().numpy()[:, :, 0, 0]
        normalized_values = f / np.sqrt(np.sum(f ** 2, -1, keepdims=True))
        activations.append(normalized_values.reshape(-1))
    return np.array(activations)

def face_perception_loss(model, img1, img2):
    
    activations_img1 = return_activations(model, img1)
    activations_img2 = return_activations(model, img2)
    loss = 0
    
    # L1 Error or L2 Error or Euclidean Distance
    for layer in range(len(activations_img1)):
        #loss += sum(np.abs(activations_img1[layer] - activations_img2[layer]))
        loss += np.sum((activations_img1[layer] - activations_img2[layer])**2)
        #loss += euc_dis(activations_img1[layer], activations_img2[layer])
    return loss

In [88]:
senet50 = initialize_senet50_2048()

low_res_img_path = '/media/angelo/DATEN/Datasets/CelebA/LR_56/test/018520.jpg'
original_img_path = '/media/angelo/DATEN/Datasets/CelebA/HR/test/018520.jpg'

lr_img = load_data(low_res_img_path)
hr_img = load_data(original_img_path)

lr_emb = return_embedding(senet50, lr_img)
hr_emb = return_embedding(senet50, hr_img)

print('Identity Loss')
print(cos_sim(lr_emb, hr_emb))
      
print('Perceptual Loss')
print(face_perception_loss(senet50, lr_img, hr_img))

Identity Loss
0.01633930206298828
Perceptual Loss
0.6450169309973717


In [44]:
img_path = '/media/angelo/DATEN/Datasets/ICB-RW/'

gallery_path = gb.glob(img_path + 'gallery/*.jpg')
gallery_names = np.array([int(os.path.splitext(name)[0][:-2]) for name in os.listdir(img_path + 'gallery/')])

resnet50 = initialize_resnet50_128()
senet50 = initialize_senet50_128()

face_embeddings = image_encoding(resnet50, gallery_path, batch_size=10)

==> compute image-level feature encoding.
-> finish encoding 90 images.


In [ ]:
# Experimentation
probe_path = img_path + 'probe/'
student_pool_list = [student for student in os.listdir(probe_path)]
watch_list_images = []
accuracy = 0

for index, student in enumerate(student_pool_list):
    print('For student ' + student)
    probe_list_students = student_pool_list.copy()
    watch_list_emb = []
    
    # First embeddings is always for the guilty student
    guilty_student_image = random.choice(os.listdir(probe_path + student)) 
    guilty_student_image = load_data(probe_path + student + '/' + guilty_student_image)
    watch_list_emb.append(return_embedding(senet50, guilty_student_image))
    
    probe_list_students.remove(student)
    
    watch_list_students = random.sample(probe_list_students, 4)
    
    for suspect in watch_list_students:
        image = random.choice(os.listdir(probe_path + suspect))
        watch_list_images.append(probe_path + suspect + '/' + image)
        
    for i, suspect in enumerate(watch_list_images):
        image = load_data(suspect)
        watch_list_emb.append(return_embedding(senet50, image))
    
    similarities = [np.abs(cos_sim(face_embeddings[index], watch_list_emb[i])) for i in range(5)]
    
    # If the least distance is the first element, let's sum 1
    if np.argmin(np.array(similarities)) == 0:
        accuracy += 1

print("Accuracy: {}".format(accuracy/len(student_pool_list)))